<a href="https://colab.research.google.com/github/TwistedAlex/TensorFlow2Practice/blob/main/TF_TUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Eager execution**
Evaluate operations immediately, without building graphs: operations *return concrete values* instead of constructing a compuational graph to run later. 
  *An intuitive interface*: Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
  *Easier debugging*: Call ops directly to inspect running models and test changes. Use standard Python debugging toools for immediate error report.
  *Natural control flow*: Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.

**Setup and basic usage**

In [1]:
import os
import tensorflow as tf
import cProfile
tf.compat.v1.enable_eager_execution()

In [2]:
# Eager execution is enabled by default in TF2.0

tf.executing_eagerly()
# Run TF ops and the results will return immediately:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))
a = tf.constant([[1, 2], [3, 4]])
print(a)
b = tf.add(a, 1)
print(b)

hello, [[4.]]
tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


Disable eager execution

In [3]:
tf.compat.v1.disable_eager_execution()
a = tf.constant([[1, 2], [3, 4]])
print(a)

Tensor("Const:0", shape=(2, 2), dtype=int32)


In [4]:
b = tf.add(a, 1)
print(b)

Tensor("Add:0", shape=(2, 2), dtype=int32)
